### Adaptive Estimator (Single Job Run)
 - Circuit Used - EfficientSU2 Echo
 - Qubit count - 5
 - EMST available - Readout EM, DD, ZNE
 - Circuit Transpilation - Default transpiled with Optimization level = 3

In [3]:
import sys
import os

# Add project root (one level up from 'notebooks')
sys.path.append(os.path.abspath(os.path.join(os.getcwd(),"../..")))

In [14]:
# kurianuthuppu11@gmail.com

token = "VybxF62Uoej4Tv3RvtlSBDeBOAryDKX97yJvsqL2gAB_"

from qiskit_ibm_runtime import QiskitRuntimeService

QiskitRuntimeService.save_account(
    token=token,
    instance="crn:v1:bluemix:public:quantum-computing:us-east:a/e3a7616824024c57b40a7014c5e27c30:b4f5e1cd-3aba-470f-93e5-f82079969538::",
    channel="ibm_quantum_platform",
    overwrite=True,
    set_as_default=True,
    name="qamp-2025",
)

In [21]:
from qiskit import QuantumCircuit
from qiskit.transpiler import generate_preset_pass_manager
from qiskit.quantum_info import SparsePauliOp
from qiskit.circuit.library import efficient_su2
from qiskit_ibm_runtime import QiskitRuntimeService, EstimatorV2 as Estimator
from adaptive_error_mitigation import adaptive_estimator
from qiskit_aer.primitives import EstimatorV2 as AerEstimator
import numpy as np


In [10]:
def gen_param4_eff_su2(n_qubits: int, seed: int = 123, reps: int = 1):
    np.random.seed(seed)
    
    # Define the bounds
    low = 0
    high = 2 * np.pi
    arr = np.random.uniform(low=low, high=high, size=n_qubits * reps * 4)
    
    return arr

def prepare_echo_eff_su2(n_qubits: int, params: list):
    crkt = efficient_su2(num_qubits=n_qubits, reps=1, entanglement="linear")
    eff_su2_crkt = crkt.assign_parameters(params)
    return eff_su2_crkt

def pm_for_backend(backend, opt_level=3, seed=123):
    pm = generate_preset_pass_manager(
        optimization_level=opt_level,
        backend = backend,
        seed_transpiler=seed
    )
    
    return pm

def create_isa_observables(n_qubits:int, isa_crkt, pauli_char="Z"):
    observable = SparsePauliOp(pauli_char * n_qubits)
    isa_obs = observable.apply_layout(isa_crkt.layout)
    
    return isa_obs

In [18]:
NUM_QUBITS = 21

params = gen_param4_eff_su2(n_qubits=NUM_QUBITS)
q_crkt = prepare_echo_eff_su2(n_qubits=NUM_QUBITS, params=params)

In [15]:
service = QiskitRuntimeService()
bknd_fez = service.backend("ibm_fez")

In [19]:
pass_mngr = pm_for_backend(bknd_fez)
isa_q_crkt = pass_mngr.run(q_crkt)

In [20]:
obs_isa = create_isa_observables(NUM_QUBITS, isa_q_crkt, "Z")

#### Running the circuit using AerEstimator

In [24]:
aer_estimator = AerEstimator()

pub = (isa_q_crkt, obs_isa)
job = aer_estimator.run([pub])
pub_result = job.result()[0]
est_val = pub_result.data.evs
est_val

array(-8.97802756e-05)

#### Running the circuit using Adaptive Estimator

In [25]:
adapt_result = adaptive_estimator.run([pub], backend=bknd_fez)


--- Processing Pub 1/1 ---
--- Initiating Adaptive Error Mitigation and Suppression Framework ---

--> DEFAULT SETTING: Using Default Precision set to 0.015625 and default shots 4096

---> HEURISTIC TRIGGERED: Readout Error Threshold Exceeded
     | Metric: MAX READOUT ERROR - 0.0415 (on Qubit 87)
     | Threshold Set: 0.0100 (READOUT_ERROR_THRESHOLD (config.py))
---> ACTION TAKEN: ENABLED Measure Mitigation (TREX)
     | Resilience Level: 1
     | Measure Noise Learning (Randomizations): 32 (NUM_RANDOMIZATIONS (config.py))
---> ACTION TAKEN: ENABLED Measure Twirling
     | Twirling: Measure=True / Gates=False
     | **Derived Parameters:** shots_per_randomization set to 128 (Shots: 4096 (DEFAULT_SHOTS (config.py)) / Randomizations: 32 (NUM_RANDOMIZATIONS (config.py)))

---> Circuit not scheduled. Applying ALAP scheduling...

---> HEURISTIC TRIGGERED: Circuit density Within Range
     | Metric: CIRCUIT DENSITY (CD) - 0.1891
     | Threshold Range: [0.0700, 0.2500] (config.py)
---> ACT

In [33]:
adapt_result[0]["est_options"]

max_execution_time,Unset
default_precision,Unset
default_shots,None
resilience_level,2
seed_estimator,Unset
experimental,Unset
▸environment,EnvironmentOptions
log_level,'WARNING'
callback,None
job_tags,None
private,False


In [34]:
job_id = adapt_result[0]["job"].job_id()

job = service.job(job_id)
pub_result = job.result()[0]
est_val_ibm = pub_result.data.evs
est_val_ibm

array(-0.02954481)

#### Running the circuit using Qiskit's Default Estimator

In [36]:
def_estimator = Estimator(mode=bknd_fez)

pub = (isa_q_crkt, obs_isa)
job = def_estimator.run([pub])
print(f"Job-Id: {job.job_id()}")

Job-Id: d5f06ucpe0pc73ajf8gg


In [37]:
pub_result = job.result()[0]
est_val_ibm_def = pub_result.data.evs
est_val_ibm_def

array(0.0077235)